In [ ]:
from cyclops.query import mimic
from cyclops.query.mimic import SUBJECT_ID

In [ ]:
events = mimic.events().run(limit=1000)

In [ ]:
from cyclops.workflow.taskprefect import normalize_events_flow

normalize_events_flow(events).result()

In [ ]:
patients = mimic.patients()
patient_diagnoses = mimic.patient_diagnoses()

In [ ]:
patients_df = patients.run()

In [ ]:
patient_diagnoses_df = patient_diagnoses.run()

In [ ]:
import time

import pandas as pd

t = time.time()
merged = pd.merge(patients_df, patient_diagnoses_df)
time.time() - t

In [ ]:
# Run queries, join in pandas
1.500441 + 25.672523 + 3.3360862731933594

In [ ]:
# Perform join in SQLAlchemy
39.795473

# Join in SQLAlchemy 12.622509
# Join in Pandas 3.336086
39.795473 - (1.500441 + 25.672523)

Check out Pandas serializer: https://docs.prefect.io/api/latest/engine/serializers.html

Override their write method for ours? Save directly instead of running/using their save

This should save some time. Especially if we save to .csv. Also, if that's where they're sorting the DataFrame, then this could be a serious time save.

Do this by defining a custom serialize function?
https://orion-docs.prefect.io/api-ref/prefect/flows/#prefect.flows.Flow.serialize_parameters

https://github.com/PrefectHQ/prefect/blob/master/src/prefect/engine/serializers.py

In [ ]:
from cyclops.query import process as qp

query = qp.Join(patients.query, on=SUBJECT_ID)(patient_diagnoses.query)
mimic.get_interface(query).run()

In [ ]:
import time

from cyclops.workflow.taskprefect import join_queries

t = time.time()
join_flow = join_queries(patient_diagnoses, patients, on=[SUBJECT_ID])
print(time.time() - t)
join_flow.result()